In [30]:
#import matplotlib.pyplot as plt
from astropy.io import fits
#from astropy.visualization import astropy_mpl_style
from astropy.wcs import WCS
import numpy as np
#plt.style.use(astropy_mpl_style)

In [2]:
%matplotlib inline

In [18]:
path1 = r'20190405T110036.fits.fz'
path2 = r'20190405T110332.fits.fz'
path3 = r'20190405T110633.fits.fz'

In [19]:
def get_imgHDU_and_wcs(path_to_fits):
    with fits.open(path_to_fits) as hdul:
        #hdul.info()
        image_hdu = hdul[1]
        wcs = WCS(image_hdu.header)
    return image_hdu,wcs
        

In [20]:
image_hdu1,wcs1 = get_imgHDU_and_wcs(path1)
image_hdu2,wcs2 = get_imgHDU_and_wcs(path2)
image_hdu3,wcs3 = get_imgHDU_and_wcs(path3)

In [21]:
reference_pixels = [[2604,1700],[2602,1712],[2580,1722],[2300,1600]] #[[x,y]] 

In [22]:
reference_world_coordinates = wcs1.all_pix2world(reference_pixels,1)
# Get pixel values in images 2 and 3 with the same coordinates
pixels_with_reference_coords_1 = wcs2.all_world2pix(reference_world_coordinates,1)
pixels_with_reference_coords_2 = wcs3.all_world2pix(reference_world_coordinates,1)

#### After the pixels with RA and DEC corresponding to reference pixels in image 1 are found, find the shift

In [26]:
def print_shift(pixels_actual,pixels_shifted):
    assert len(pixels_actual)==len(pixels_shifted),"Ensure inputs have same length"
    for ii in range(len(pixels_actual)):
        x_shift,y_shift = pixels_actual[ii]-pixels_shifted[ii]
        print("\nx shift is {0} pixels and y shift is {1} pixels".format(x_shift,y_shift))

In [29]:
print('Between image 1 and image 2')
print_shift(reference_pixels,pixels_with_reference_coords_1)
print('\nBetween image 1 and image 3')
print_shift(reference_pixels,pixels_with_reference_coords_1)

Between image 1 and image 2

x shift is 0.07305847275119959 pixels and y shift is -3.115030300623175 pixels

x shift is 0.07194780935969902 pixels and y shift is -3.116035684471399 pixels

x shift is 0.07229725723345837 pixels and y shift is -3.1191111032048866 pixels

x shift is 0.10141022413881728 pixels and y shift is -3.1401492937727653 pixels

Between image 1 and image 3

x shift is 0.07305847275119959 pixels and y shift is -3.115030300623175 pixels

x shift is 0.07194780935969902 pixels and y shift is -3.116035684471399 pixels

x shift is 0.07229725723345837 pixels and y shift is -3.1191111032048866 pixels

x shift is 0.10141022413881728 pixels and y shift is -3.1401492937727653 pixels


It seems the greater the distance from the center, greater is the shift. 